# Convolutional Network in TensorFlow

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [ ]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

In [3]:
learning_rate = 0.00001
epochs = 10
batch_size = 128

test_valid_size = 256
n_class = 10 # MNIST total classes (0-9)
dropout = 0.75

weights = {
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024,n_class]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_class]))
}

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding='SAME')

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28x28x1 to 14x14x32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer 2 - 14x14x32 to 7x7x64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer - 7x7x64 to 1024
    fc1 = tf.reshape(conv2, [-1,weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output Layer - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [7]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_class])
keep_prob = tf.placeholder(tf.float32)

logits = conv_net(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout
            })
            
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.
            })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                  epoch + 1,
                  batch + 1,
                  loss,
                  valid_acc))
            
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.
    })
    
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 41302.7539 Validation Accuracy: 0.097656
Epoch  1, Batch   2 -Loss: 42873.3516 Validation Accuracy: 0.113281
Epoch  1, Batch   3 -Loss: 36220.6836 Validation Accuracy: 0.117188
Epoch  1, Batch   4 -Loss: 31428.6680 Validation Accuracy: 0.121094
Epoch  1, Batch   5 -Loss: 31872.1016 Validation Accuracy: 0.121094
Epoch  1, Batch   6 -Loss: 28592.3320 Validation Accuracy: 0.132812
Epoch  1, Batch   7 -Loss: 26107.9727 Validation Accuracy: 0.136719
Epoch  1, Batch   8 -Loss: 23244.2637 Validation Accuracy: 0.136719
Epoch  1, Batch   9 -Loss: 24199.0391 Validation Accuracy: 0.121094
Epoch  1, Batch  10 -Loss: 23110.9277 Validation Accuracy: 0.140625
Epoch  1, Batch  11 -Loss: 15887.0244 Validation Accuracy: 0.136719
Epoch  1, Batch  12 -Loss: 22897.2734 Validation Accuracy: 0.140625
Epoch  1, Batch  13 -Loss: 19149.4570 Validation Accuracy: 0.183594
Epoch  1, Batch  14 -Loss: 18277.6992 Validation Accuracy: 0.195312
Epoch  1, Batch  15 -Loss: 17507.7422 Validation

Epoch  1, Batch 126 -Loss:  3104.6853 Validation Accuracy: 0.554688
Epoch  1, Batch 127 -Loss:  4227.7339 Validation Accuracy: 0.566406
Epoch  1, Batch 128 -Loss:  3812.1885 Validation Accuracy: 0.558594
Epoch  1, Batch 129 -Loss:  3728.2417 Validation Accuracy: 0.558594
Epoch  1, Batch 130 -Loss:  4033.5151 Validation Accuracy: 0.578125
Epoch  1, Batch 131 -Loss:  3858.0876 Validation Accuracy: 0.566406
Epoch  1, Batch 132 -Loss:  3833.8477 Validation Accuracy: 0.570312
Epoch  1, Batch 133 -Loss:  3639.9834 Validation Accuracy: 0.582031
Epoch  1, Batch 134 -Loss:  4371.7124 Validation Accuracy: 0.570312
Epoch  1, Batch 135 -Loss:  2610.9131 Validation Accuracy: 0.574219
Epoch  1, Batch 136 -Loss:  3292.5183 Validation Accuracy: 0.566406
Epoch  1, Batch 137 -Loss:  3637.1616 Validation Accuracy: 0.562500
Epoch  1, Batch 138 -Loss:  3625.8335 Validation Accuracy: 0.566406
Epoch  1, Batch 139 -Loss:  3982.2920 Validation Accuracy: 0.562500
Epoch  1, Batch 140 -Loss:  3972.1870 Validation

Epoch  1, Batch 257 -Loss:  1910.4467 Validation Accuracy: 0.675781
Epoch  1, Batch 258 -Loss:  2356.9736 Validation Accuracy: 0.691406
Epoch  1, Batch 259 -Loss:  1849.5210 Validation Accuracy: 0.691406
Epoch  1, Batch 260 -Loss:  1974.5973 Validation Accuracy: 0.691406
Epoch  1, Batch 261 -Loss:  2013.7313 Validation Accuracy: 0.691406
Epoch  1, Batch 262 -Loss:  1714.1128 Validation Accuracy: 0.687500
Epoch  1, Batch 263 -Loss:  2045.1239 Validation Accuracy: 0.691406
Epoch  1, Batch 264 -Loss:  2185.4670 Validation Accuracy: 0.687500
Epoch  1, Batch 265 -Loss:  1635.3080 Validation Accuracy: 0.683594
Epoch  1, Batch 266 -Loss:  2273.2168 Validation Accuracy: 0.687500
Epoch  1, Batch 267 -Loss:  2436.2551 Validation Accuracy: 0.691406
Epoch  1, Batch 268 -Loss:  2893.8203 Validation Accuracy: 0.679688
Epoch  1, Batch 269 -Loss:  1736.9258 Validation Accuracy: 0.679688
Epoch  1, Batch 270 -Loss:  1615.4294 Validation Accuracy: 0.675781
Epoch  1, Batch 271 -Loss:  2846.2502 Validation

Epoch  1, Batch 386 -Loss:  1765.1716 Validation Accuracy: 0.710938
Epoch  1, Batch 387 -Loss:  1856.2731 Validation Accuracy: 0.710938
Epoch  1, Batch 388 -Loss:  2005.2434 Validation Accuracy: 0.710938
Epoch  1, Batch 389 -Loss:  2012.6155 Validation Accuracy: 0.707031
Epoch  1, Batch 390 -Loss:  1555.9337 Validation Accuracy: 0.710938
Epoch  1, Batch 391 -Loss:  1651.9851 Validation Accuracy: 0.707031
Epoch  1, Batch 392 -Loss:  1307.1133 Validation Accuracy: 0.714844
Epoch  1, Batch 393 -Loss:  1877.8224 Validation Accuracy: 0.703125
Epoch  1, Batch 394 -Loss:  1867.4436 Validation Accuracy: 0.703125
Epoch  1, Batch 395 -Loss:  2049.8389 Validation Accuracy: 0.703125
Epoch  1, Batch 396 -Loss:  2099.7307 Validation Accuracy: 0.707031
Epoch  1, Batch 397 -Loss:  1161.5848 Validation Accuracy: 0.714844
Epoch  1, Batch 398 -Loss:  1146.6558 Validation Accuracy: 0.707031
Epoch  1, Batch 399 -Loss:  1785.6584 Validation Accuracy: 0.703125
Epoch  1, Batch 400 -Loss:  1967.3104 Validation

Epoch  2, Batch  81 -Loss:  1720.3904 Validation Accuracy: 0.730469
Epoch  2, Batch  82 -Loss:  1503.4680 Validation Accuracy: 0.734375
Epoch  2, Batch  83 -Loss:  1096.4888 Validation Accuracy: 0.726562
Epoch  2, Batch  84 -Loss:  1099.9766 Validation Accuracy: 0.738281
Epoch  2, Batch  85 -Loss:  1540.8358 Validation Accuracy: 0.738281
Epoch  2, Batch  86 -Loss:  1318.9183 Validation Accuracy: 0.738281
Epoch  2, Batch  87 -Loss:  2000.0562 Validation Accuracy: 0.734375
Epoch  2, Batch  88 -Loss:  1116.8665 Validation Accuracy: 0.734375
Epoch  2, Batch  89 -Loss:  1181.9062 Validation Accuracy: 0.734375
Epoch  2, Batch  90 -Loss:  1142.2555 Validation Accuracy: 0.734375
Epoch  2, Batch  91 -Loss:   783.3693 Validation Accuracy: 0.726562
Epoch  2, Batch  92 -Loss:   920.4265 Validation Accuracy: 0.726562
Epoch  2, Batch  93 -Loss:  1197.2463 Validation Accuracy: 0.734375
Epoch  2, Batch  94 -Loss:  1014.1817 Validation Accuracy: 0.730469
Epoch  2, Batch  95 -Loss:  1405.3264 Validation

...

Epoch  9, Batch 275 -Loss:   171.0235 Validation Accuracy: 0.777344
Epoch  9, Batch 276 -Loss:   339.7914 Validation Accuracy: 0.773438
Epoch  9, Batch 277 -Loss:   276.2766 Validation Accuracy: 0.773438
Epoch  9, Batch 278 -Loss:   341.5316 Validation Accuracy: 0.773438
Epoch  9, Batch 279 -Loss:   306.0646 Validation Accuracy: 0.773438
Epoch  9, Batch 280 -Loss:   350.3018 Validation Accuracy: 0.769531
Epoch  9, Batch 281 -Loss:   274.4093 Validation Accuracy: 0.765625
Epoch  9, Batch 282 -Loss:   392.2444 Validation Accuracy: 0.765625
Epoch  9, Batch 283 -Loss:   256.8494 Validation Accuracy: 0.765625
Epoch  9, Batch 284 -Loss:   267.7184 Validation Accuracy: 0.761719
Epoch  9, Batch 285 -Loss:   208.3067 Validation Accuracy: 0.761719
Epoch  9, Batch 286 -Loss:   226.2882 Validation Accuracy: 0.761719
Epoch  9, Batch 287 -Loss:   248.1048 Validation Accuracy: 0.761719
Epoch  9, Batch 288 -Loss:   192.8168 Validation Accuracy: 0.761719
Epoch  9, Batch 289 -Loss:   248.5429 Validation

Epoch  9, Batch 404 -Loss:   245.5412 Validation Accuracy: 0.773438
Epoch  9, Batch 405 -Loss:   262.8192 Validation Accuracy: 0.773438
Epoch  9, Batch 406 -Loss:   294.0351 Validation Accuracy: 0.761719
Epoch  9, Batch 407 -Loss:   280.2307 Validation Accuracy: 0.769531
Epoch  9, Batch 408 -Loss:   220.9796 Validation Accuracy: 0.765625
Epoch  9, Batch 409 -Loss:   264.8595 Validation Accuracy: 0.761719
Epoch  9, Batch 410 -Loss:   318.1503 Validation Accuracy: 0.765625
Epoch  9, Batch 411 -Loss:   336.4654 Validation Accuracy: 0.765625
Epoch  9, Batch 412 -Loss:   216.5821 Validation Accuracy: 0.765625
Epoch  9, Batch 413 -Loss:   224.2040 Validation Accuracy: 0.765625
Epoch  9, Batch 414 -Loss:   300.5556 Validation Accuracy: 0.769531
Epoch  9, Batch 415 -Loss:   275.6018 Validation Accuracy: 0.765625
Epoch  9, Batch 416 -Loss:   333.3669 Validation Accuracy: 0.765625
Epoch  9, Batch 417 -Loss:   221.3448 Validation Accuracy: 0.761719
Epoch  9, Batch 418 -Loss:   224.5227 Validation

Epoch 10, Batch 102 -Loss:   213.8551 Validation Accuracy: 0.757812
Epoch 10, Batch 103 -Loss:   264.0718 Validation Accuracy: 0.757812
Epoch 10, Batch 104 -Loss:   269.3796 Validation Accuracy: 0.757812
Epoch 10, Batch 105 -Loss:   285.1419 Validation Accuracy: 0.757812
Epoch 10, Batch 106 -Loss:   170.2813 Validation Accuracy: 0.765625
Epoch 10, Batch 107 -Loss:   254.3764 Validation Accuracy: 0.761719
Epoch 10, Batch 108 -Loss:   339.5673 Validation Accuracy: 0.757812
Epoch 10, Batch 109 -Loss:   236.3896 Validation Accuracy: 0.765625
Epoch 10, Batch 110 -Loss:   285.0840 Validation Accuracy: 0.769531
Epoch 10, Batch 111 -Loss:   200.6071 Validation Accuracy: 0.769531
Epoch 10, Batch 112 -Loss:   232.0998 Validation Accuracy: 0.757812
Epoch 10, Batch 113 -Loss:   201.5205 Validation Accuracy: 0.757812
Epoch 10, Batch 114 -Loss:   192.3312 Validation Accuracy: 0.761719
Epoch 10, Batch 115 -Loss:   231.8901 Validation Accuracy: 0.757812
Epoch 10, Batch 116 -Loss:   297.9070 Validation

Epoch 10, Batch 233 -Loss:   288.9472 Validation Accuracy: 0.765625
Epoch 10, Batch 234 -Loss:   294.5522 Validation Accuracy: 0.765625
Epoch 10, Batch 235 -Loss:   238.6017 Validation Accuracy: 0.761719
Epoch 10, Batch 236 -Loss:   285.9841 Validation Accuracy: 0.773438
Epoch 10, Batch 237 -Loss:   283.4760 Validation Accuracy: 0.761719
Epoch 10, Batch 238 -Loss:   221.2871 Validation Accuracy: 0.761719
Epoch 10, Batch 239 -Loss:   295.2018 Validation Accuracy: 0.773438
Epoch 10, Batch 240 -Loss:   228.6704 Validation Accuracy: 0.777344
Epoch 10, Batch 241 -Loss:   302.7867 Validation Accuracy: 0.773438
Epoch 10, Batch 242 -Loss:   166.8869 Validation Accuracy: 0.773438
Epoch 10, Batch 243 -Loss:   215.6851 Validation Accuracy: 0.773438
Epoch 10, Batch 244 -Loss:   301.8167 Validation Accuracy: 0.769531
Epoch 10, Batch 245 -Loss:   168.3653 Validation Accuracy: 0.765625
Epoch 10, Batch 246 -Loss:   171.4942 Validation Accuracy: 0.765625
Epoch 10, Batch 247 -Loss:   256.8015 Validation

Epoch 10, Batch 364 -Loss:   256.6743 Validation Accuracy: 0.761719
Epoch 10, Batch 365 -Loss:   229.0934 Validation Accuracy: 0.757812
Epoch 10, Batch 366 -Loss:   228.6481 Validation Accuracy: 0.765625
Epoch 10, Batch 367 -Loss:   201.9316 Validation Accuracy: 0.757812
Epoch 10, Batch 368 -Loss:   237.0962 Validation Accuracy: 0.761719
Epoch 10, Batch 369 -Loss:   279.9734 Validation Accuracy: 0.765625
Epoch 10, Batch 370 -Loss:   292.2431 Validation Accuracy: 0.765625
Epoch 10, Batch 371 -Loss:   277.5484 Validation Accuracy: 0.765625
Epoch 10, Batch 372 -Loss:   210.0190 Validation Accuracy: 0.765625
Epoch 10, Batch 373 -Loss:   196.5131 Validation Accuracy: 0.765625
Epoch 10, Batch 374 -Loss:   233.0790 Validation Accuracy: 0.769531
Epoch 10, Batch 375 -Loss:   255.4492 Validation Accuracy: 0.773438
Epoch 10, Batch 376 -Loss:   305.9332 Validation Accuracy: 0.773438
Epoch 10, Batch 377 -Loss:   319.2129 Validation Accuracy: 0.777344
Epoch 10, Batch 378 -Loss:   184.1536 Validation